<a href="https://colab.research.google.com/github/DanialPahlavan/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# intro
Basic RAG from article Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

https://arxiv.org/abs/2005.11401

## KoRC from THU-KEG

https://github.com/THU-KEG/KoRC/tree/main

In [ ]:
!git clone https://github.com/THU-KEG/KoRC.git

Cloning into 'KoRC'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 132 (delta 49), reused 119 (delta 41), pack-reused 0
Receiving objects: 100% (132/132), 85.80 KiB | 4.77 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
!wget https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz

--2024-05-18 14:00:49--  https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.108, 3.163.189.14, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4694541059 (4.4G) [application/gzip]
Saving to: ‘psgs_w100.tsv.gz’

psgs_w100.tsv.gz    100%[===================>]   4.37G   100MB/s    in 23s     

2024-05-18 14:01:12 (191 MB/s) - ‘psgs_w100.tsv.gz’ saved [4694541059/4694541059]



In [ ]:
!gzip -d psgs_w100.tsv.gz

In [ ]:
!pwd

/content


### preprocess

#### small DB

In [ ]:
from tqdm import tqdm

# Open the file with a context manager to ensure it's closed properly
with open('psgs_w100.tsv', 'r') as file:
    new_lines = []
    # Use enumerate for counting lines and tqdm for progress bar
    for i, line in enumerate(tqdm(file), start=1):
        # Skip the first line (header)
        if i == 1:
            continue
        # Break the loop if 500 lines have been processed
        if i > 500:
            break
        id, text, title = line.strip().split("\t")
        new_lines.append(title + '\t' + text + '\n')

# Specify the full path to the file
with open('/content/KoRC/RAG/preprocess/data.csv', 'w') as f:
    f.writelines(new_lines)


500it [00:00, 111248.85it/s]


### full DB
just use small or full not both

In [ ]:
from tqdm import tqdm

new_lines = []
for line in tqdm(open('psgs_w100.tsv').readlines()[1:]):
    id, text, title = line.strip().split("\t")
    new_lines.append(title+'\t'+text+'\n')

# Specify the full path to the file
with open('/content/KoRC/RAG/preprocess/data.csv', 'w') as f:
    f.writelines(new_lines)


In [ ]:
!python /content/KoRC/RAG/preprocess/convert.py

^C


### build_faiss

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss-gpu
!python -m pip install --upgrade faiss-cpu

#gpu
#!pip install faiss-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.6 MB/s eta 0:00:00


In [ ]:
!python /content/KoRC/RAG/preprocess/build_faiss.py --csv_path /content/KoRC/RAG/preprocess/data.csv

INFO:__main__:Step 1 - Create the dataset
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

### prepare_open_dataset.py

In [ ]:
!python /content/KoRC/RAG/prepare_openqa_dataset.py

Traceback (most recent call last):
  File "/content/KoRC/RAG/prepare_openqa_dataset.py", line 35, in <module>
    mrc_train = json.load(open(os.path.join(input_path,'train.json')))
  File "/usr/lib/python3.10/posixpath.py", line 76, in join
    a = os.fspath(a)
TypeError: expected str, bytes or os.PathLike object, not NoneType


In [ ]:
!python prepare_feature.py

In [ ]:
!bash seq.human.sh